# Tutorial 3: A First Model

After getting a basic understanding of the data in the previous tutorial, we are ready to build a first AI model.
In this notebook, we build and evaluate a simple model for identifying worm sections in nodule images.

By the end of this notebook you will have created a first submission that you can submit to put your team on the leaderboard!

**NOTE: This notebook will only work with a GPU instance. Make sure that you selected *ml.g4dn.xlarge* as instance type**

# Picking the right approach

The first challenge we have to address is how to even tackle the problem. What could be a good approach / algorithm? How are other people doing it?
[PapersWithCode](https://paperswithcode.com/sota) is an amazing resource for exactly these questions. It lists typical problem classes together with the current state-of-the-art and links to popular frameworks.
For our problem, [Object Detection](https://paperswithcode.com/task/object-detection) is the right category. If you follow the link you'll find a list of popular frameworks.

[MMDetection](https://github.com/open-mmlab/mmdetection) is one of the most widely used and the one we picked for this tutorial. Not only does it provide a great [tutorial](https://github.com/open-mmlab/mmdetection/blob/master/demo/MMDet_Tutorial.ipynb), but also offers a variety of [different architectures and settings](https://github.com/open-mmlab/mmdetection#overview-of-benchmark-and-model-zoo) that you can play around with.

In this notebook, we will adapt the MMdetection [tutorial](https://github.com/open-mmlab/mmdetection/blob/master/demo/MMDet_Tutorial.ipynb) to our problem and run it on a toy dataset.

For those familiar with deep learning architectures, MMDetection can handle data loading, preperation, model building, and training by altering a few common settings in the configuration file. Data preperation also includes various data augmentation techinques like rotation or flipping. The majority of the most common CV arcitectures are already build and available via configuration.

### *Further reading:*
Going into details of object detection is beyond the scope of this tutorial. If you'd like to know more, here are two recommendations to get started:
- [An introduction to R-CNNs and object detection in general](https://towardsdatascience.com/deep-dive-into-the-computer-vision-world-part-2-7a24efdb1a14)
- [An graphic explanation how convolutional neural networks work](https://towardsdatascience.com/gentle-dive-into-math-behind-convolutional-neural-networks-79a07dd44cf9)

# Setup
As in the other notebooks, we start by importing the relevant libraries and global settings.

In [2]:
import matplotlib.pyplot as plt  # Used for plotting
import mmcv  # Object detection framework
import os  # Interaction with the file system
import pandas as pd  # Home of the DataFrame construct, _the_ most important object for Data Science
import sys  # Python system library needed to load custom functions

from matplotlib.patches import Rectangle  # Allows drawing the bounding boxes of the worm sections
from mmcv import Config  # Loading and accessing MMDetection configuration files
from mmdet.apis import inference_detector, init_detector, train_detector, set_random_seed  # Part of the MMDetection framework
from mmdet.datasets import build_dataset  # Part of the MMDetection framework
from mmdet.models import build_detector  # Part of the MMDetection framework

from PIL import Image  # For loading image files
from tqdm import tqdm  # for timing a for loop

In [3]:
sys.path.append('../src')  # Add the source directory to the PYTHONPATH. This allows to import local functions and modules.

In [4]:
from Dataset import OnchoDataset
from detection_util import create_predictions
from gdsc_score import get_leaderboard_score
from gdsc_util import download_directory, download_file, load_sections_df, set_up_logging, PROJECT_DIR
from PredictionEvaluator import PredictionEvaluator

set_up_logging()  # Sets up logging to console and the .log file
data_folder = str(PROJECT_DIR / 'data')

# Preparing the Training Dataset

Whenever you start working with a new framework you can expect issues with the setup.
To ensure that we can quickly try things out we create a small subset of our data.

In [5]:
section_df = load_sections_df(f'{data_folder}/gdsc_train.csv')
len(section_df)

65687

Our complete dataset has more than 65.000 rows. Let's create two small dummy sets with 100 and 50 entries only.

In [6]:
dummy_train = section_df[:100]
dummy_test = section_df[100:150]
dummy_train.to_csv(f'{data_folder}/dummy_train.csv', sep=';')
dummy_test.to_csv(f'{data_folder}/dummy_test.csv', sep=';')

We store the reduced sets under *../data*. Verify that you can find and open the files there. 
To make sure that everything works, let's load the files again.

In [7]:
dummy_train = load_sections_df(f'{data_folder}/dummy_train.csv')
dummy_train.head()

,file_name,study,staining,xmin,xmax,ymin,ymax,height,width
section_id,,,,,,,,,
1_D@2317-5556-2836-6232,1_D.jpg,Study_1,D,2317,2836,5556,6232,8192,7380
1_D@2407-6156-2952-6789,1_D.jpg,Study_1,D,2407,2952,6156,6789,8192,7380
1_D@2483-5836-4279-7287,1_D.jpg,Study_1,D,2483,4279,5836,7287,8192,7380
1_D@2546-1211-4374-2558,1_D.jpg,Study_1,D,2546,4374,1211,2558,8192,7380
1_D@2695-5530-3238-6179,1_D.jpg,Study_1,D,2695,3238,5530,6179,8192,7380


Looks good! 

# Creating a MMDetection Configuration File

MMDetection relies on extensive configuration files. The usual process is to adapt an already existing configuration file and apply transfer learning.
To do this, we first need to download the configuration and weights files.

In [8]:
# Make a new folder in the data folder
!mkdir ../data/checkpoints 
# Install wget, a program for downloading files from the internet
!apt update
!apt install wget
# Download the config and weights file
!wget -c https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth -O ../data/checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth

mkdir: cannot create directory ‘../data/checkpoints’: File exists
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                
Hit:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease              
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease              
Reading package lists... Done                                          
Building dependency tree       
Reading state information... Done
103 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
0 upgraded, 0 newly installed, 0 to remove and 103 not upgraded.
--2022-07-12 06:54:49--  https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_

The configuration file is part of the mmdetection package that is already installed on this image. Hence we can load our test configuration via

In [9]:
cfg = Config.fromfile('/mmdetection/configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py')

The configuration is a dictionary-like object that stores our configuration settings. Is has [a lot](https://mmdetection.readthedocs.io/en/latest/tutorials/config.html) of settings and with the right options, you can run the [really advanced models](https://github.com/open-mmlab/mmdetection/blob/master/docs/en/model_zoo.md).
Below we modify some of those settings to point the model to our train and test data.

We start by defining the structure and location of our data.

In [10]:
# Modify dataset type and path
cfg.dataset_type = 'OnchoDataset' #this is a custom data loader script we created for the GDSC you can view it in src/Dataset.py
cfg.data_root = data_folder

cfg.data.train.type = 'OnchoDataset'
cfg.data.train.data_root = data_folder      # path to the folder data
cfg.data.train.img_prefix = 'jpgs/'         # path from data_root to the images folder
cfg.data.train.ann_file = 'dummy_train.csv' # the file containing the train data labels

cfg.data.test.type = 'OnchoDataset'
cfg.data.test.data_root = data_folder
cfg.data.test.img_prefix = 'jpgs/'
cfg.data.test.ann_file = 'dummy_test.csv'

cfg.data.val.type = 'OnchoDataset'          # We will not use a separate validation data set in this tutorial, but we need to specify the values to overwrite the COCO defaults.
cfg.data.val.data_root = data_folder
cfg.data.val.img_prefix = 'jpgs/'
cfg.data.val.ann_file = 'dummy_test.csv'

We specify where to save the results and where the weights we will use were downloaded to. We also set the number of workers.

In [11]:
# We can still use the pre-trained Mask RCNN model though we do not need to use the mask branch
cfg.load_from = f'{data_folder}/checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'

# Set up working dir to save files and logs.
cfg.work_dir = f'{data_folder}/tutorial_exps/'
# Ensure work_dir exists
mmcv.mkdir_or_exist(os.path.abspath(cfg.work_dir))

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

cfg.data.samples_per_gpu = 3 # These numbers will change depending on the size of your model and GPU.
cfg.data.workers_per_gpu = 1 # These values are what we have found to be best for this model and GPU

cfg.device = 'cuda' # This defines that we will use a GPU

Next are the model settings. One thing we have to change is the number of classes to predict. Since we only have one class (worm section), we need to set this to one.
We also change the learning rate since we only have one GPU.

In [12]:
# modify number of classes of the model in box head
cfg.model.roi_head.bbox_head.num_classes = 1  # a worm section is the only object we are detecting

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU and multiply by the number of GPU workers.
cfg.optimizer.lr = 0.02 / 8 * cfg.data.workers_per_gpu
cfg.lr_config.warmup = None
cfg.log_config.interval = 10

Finally, we specify how and when to evaluate the model and how long to train the model. We will use the [mAP](https://blog.paperspace.com/mean-average-precision/#:~:text=To%20evaluate%20object%20detection%20models,model%20is%20in%20its%20detections.) metric to evaluate how the training is going.

For our test run, we will only train for three epochs. For a real training run you'll want to set this number higher.
After each epoch, the current weights will be saved in the *work_dir* folder we set above. Also the model will be evaluated on the test data set.
This allows us to check if we should continue training or stop.

In [13]:
# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1
# How long do we want to train
cfg.runner.max_epochs = 3

# Training of the first Model

We're now ready to train a first simple model! The first step is to prepare the dataset.

In [14]:
datasets = [build_dataset(cfg.data.train)]

datasets

2022-07-12 06:54:50,202 - Dataset - INFO - Building Dataset
2022-07-12 06:54:50,202 - Dataset - INFO - Building Dataset


/mmdetection/mmdet/datasets/custom.py:180: UserWarning: CustomDataset does not support filtering empty gt images.
  'CustomDataset does not support filtering empty gt images.')


[
 OnchoDataset Train dataset with number of images 3, and instance counts: 
 +-------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
 | category    | count | category | count | category | count | category | count | category | count |
 +-------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+
 |             |       |          |       |          |       |          |       |          |       |
 | 0 [section] | 100   |          |       |          |       |          |       |          |       |
 +-------------+-------+----------+-------+----------+-------+----------+-------+----------+-------+]

The dataset contains exactly 100 worm sections, one for each row on the *dummy_train* DataFrame. We also get a warning about filtering empty ground truth images which we can ignore since we don't have those.

Next, we initialize the model according to the configuration. *train_cfg* determins on which data to train the model, *test_cfg* on which data to test the model.

In [18]:
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
model.CLASSES = datasets[0].CLASSES  # Add an attribute for visualization convenience

And finally, we can train the model. The log output helps us understand how well the model is doing. 
Since we set the *evaluation.interval* to *1*, the model will be evaluated after every epoch. In general you want to continue training as long as the *mAP* score increases.

In [19]:
train_detector(model, datasets, cfg, validate=True)

2022-07-12 06:57:51,671 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-07-12 06:57:51,671 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-07-12 06:57:51,688 - Dataset - INFO - Building Dataset
2022-07-12 06:57:51,688 - Dataset - INFO - Building Dataset
2022-07-12 06:57:51,701 - mmdet - INFO - load checkpoint from local path: /home/sagemaker-user/gdsc5-tutorials-public/data/checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth
2022-07-12 06:57:51,701 - mmdet - INFO - load checkpoint from local path: /home/sagemaker-user/gdsc5-tutorials-public/data/checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth
2022-07-12 06:57:52,345 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) fr

RuntimeError: DataLoader worker (pid(s) 1028) exited unexpectedly

The log starts with a warning about a size mismatch that we can ignore. It is due to the fact that the weights we load belong to a model that was trained on the COCO dataset which has 80 different classes whereas we only have one class to detect.

Since the *mAP* keep increasing it would make sense to continue training the model for longer. 
But before we do that let's make sure that the rest of the process works.

**Note**: 
- **If you get an error, try rerunning the notebook starting with the loading of the config.** 
- **If you get the error *RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 7, 7], but got 5-dimensional input of size [1, 3, 3, 928, 768] instead* change the instance to one with a GPU.** 



# Running the Model

After training the model we can now apply it on a new image and look at the predictions. 
For this, we first load the image and initalize the model from a stored checkpoint.

In [17]:
example_image = '1_D.jpg'
img =  mmcv.imread(f'{data_folder}/jpgs/{example_image}')

In [ ]:
checkpoint = f'{cfg.work_dir}epoch_3.pth' # Select one of the model checkpoints to load in
model = init_detector(cfg, checkpoint, device='cuda')

Run the inference

In [ ]:
detections = inference_detector(model, img)
print(len(detections))  
detections[0][:2]  # Show the first two entries

*detections* is a list of all the detected worm sections. Every detection is a 5-tuple of the form *(xmin, ymin, xmax, ymax, score)*, i.e. the location of the bounding box followed by the confidence.

**Exercise:**
- Plot the predicted worm boxes. You may use the code provided in tutorial 2.
- Load the model weights of a different checkpoint, e.g. after the first epoch. What are the differences?

# Evaluating the model

To get an idea how well we're doing we can run the same evaluation function that will be used for the leaderboard on our example file.
For this we first need to convert to model output into the format that is readable by our scoring function.

In [ ]:
columns = ['section_id', 'file_name', 'xmin', 'xmax', 'ymin', 'ymax', 'detection_score']
df_results = pd.DataFrame(columns=columns)

predictions = []

# Loop over all detected boxes and convert the information into a dictionary
for box in detections[0]:
    xmin, ymin, xmax, ymax, score = box
    xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)  # Convert predicted coordinates to integer values
    box_dict = dict(
        section_id=f'{example_image}@{xmin}-{xmax}-{ymin}-{ymax}',
        file_name=example_image,
        xmin=xmin,
        ymin=ymin,
        xmax=xmax,
        ymax=ymax,
        detection_score=score
    )
    predictions.append(box_dict)

# Convert the dictionary into a dataframe with section_id as index
prediction_df = pd.DataFrame(predictions)
prediction_df.set_index('section_id', inplace=True)
prediction_df.head()

Next, we restrict our dataset to only the file for which we made predictions (*1_D.jpg*) and run the score function

In [ ]:
ground_truth = section_df.loc[section_df.file_name==example_image]

In [ ]:
evaluator = PredictionEvaluator(ground_truth)
thresholds = [0.5, 0.6, 0.7]

For the leaderboard, the predicted worm section boxes are compared to the actual worm section boxes. We compute the [Intersection over Union (IOU)](https://towardsdatascience.com/iou-a-better-detection-evaluation-metric-45a511185be1) between the boxes. If the value is greater than a threshold (e.g. 0.5) the prediction counts as a match.

In [ ]:
get_leaderboard_score(prediction_df, thresholds, evaluator)

The function *get_leaderboard_score* gives us a lot of information that we can use to debug our model. 
- *score* is the score that will be shown in the leaderboard. It is the sum of the individual detection_acc@X values.
- Additionally, for each IOU threshold the *accuracy, true positives, false positives* and *false negatives* are computed.

**Exercise:**
- Which type of error is the most common? How could you combat this?
- The output shows true positives (tp), false positives (fp), false negatives (fn) but no true negatives. Why is that?
- Evaluate the model on the *dummy_test* dataset we created previously.

# A first submission

Now, we have all the pieces in place to create our first submission. We will predict the worm sections for all files and store them in a DataFrame.
We will need to provide a prediction for all the files that are listed in *test_files.csv*. Let's load the file first.

In [ ]:
files = pd.read_csv(f'{data_folder}/test_files.csv', sep=';', header=None)
file_names = files[0].values
file_names

For simplicity, we created a function that runs the above code on all file names and returns a dataframe with the predictions. This will take around five minutes.

In [ ]:
prediction_df = create_predictions(file_names, cfg, checkpoint, device='cuda')

In [ ]:
prediction_df.head()

Looks good! All that's left is to save the results to a csv and upload it on the [GDSC website](https://gdsc.ce.capgemini.com/).

In [ ]:
prediction_df.to_csv(f'{data_folder}/results_tutorial3.csv', sep=';')

This submission should lead to a score of around 28. 

**Exercise:**
- The submission score is a lot less than the score of 70.07 we computed above. What are potential reasons for this?

# Summary

This notebook covered a LOT of different topics. We covered how to 
- Train a basic object detection model
- Use a trained model to create predictions on a nodule image
- How to evaluate the predictions
- How to create a first submission

all done on a dummy dataset. With this, we have the technical foundations to creating a good submission!

In the next tutorial, you will learn how to run and evaluate the model from this notebook on the complete dataset as a Sagemaker training job.

**REMINDER: Remember to shut down the *ml.g4dn.xlarge* instance when you aren't using it.**